# **09** Exceptions vs. Options and More Application Operators

We've implemented stacks and queues with lists now! The only difference in our data structures themselves is our use of exceptions and options.

In [1]:
module ListStack = struct
	type 'a stack = 'a list
	
	let empty = []
	
	let push x s = x :: s
	
	let peek = function
		| [] -> failwith "Empty"
		| x :: _ -> x
	
	let pop = function
		| [] -> failwith "Empty"
		| _ :: s -> s
	
end

module ListQueue = struct
	type 'a queue = 'a list
	
	let empty = []
	
	(* use the append operator to put it at the end of the list *)
	(* linear time :'( *)
	let enqueue x q = q @ [x]
	
	let dequeue = function
		| [] -> None
		| _ :: t -> Some t
	
	let peek = function
		| [] -> None
		| h :: _ -> Some h
end

module ListStack :
  sig
    type 'a stack = 'a list
    val empty : 'a list
    val push : 'a -> 'a list -> 'a list
    val peek : 'a list -> 'a
    val pop : 'a list -> 'a list
  end


module ListQueue :
  sig
    type 'a queue = 'a list
    val empty : 'a list
    val enqueue : 'a -> 'a list -> 'a list
    val dequeue : 'a list -> 'a list option
    val peek : 'a list -> 'a option
  end


Exceptions make it easy to pipeline operators! You can just keep adding operations as you go.

In [2]:
ListStack.(empty |> push 42 |> pop |> push 43)

- : int list = [43]


Options make that a bit trickier.

In [3]:
ListQueue.(empty |> enqueue 42 |> dequeue |> enqueue 43)

error: compile_error

 Uh oh... We get a type checking error because `dequeue` returned an option, while `enqueue` expected an `int list`, not an `int list option`.
 
 One way to fix this would be to write a new pipeline operator to deal with this. Recall the definition of the pipeline operator:
 ```ocaml
 let ( |> ) x f = f x
 ```

In [4]:
let ( >>| ) opt f =
	match opt with
		| None -> None
		| Some x -> Some (f x)

val ( >>| ) : 'a option -> ('a -> 'b) -> 'b option = <fun>


This is under the standard library as `Option.map`.

In [5]:
ListQueue.(empty |> enqueue 42 |> dequeue >>| enqueue 43)

- : int list option = Some [43]


What if we wanted to dequeue after that?

In [6]:
ListQueue.(empty |> enqueue 42 |> dequeue >>| enqueue 43 |> dequeue)

error: compile_error

In [7]:
ListQueue.(empty |> enqueue 42 |> dequeue >>| enqueue 43 >>| dequeue)

- : int list option option = Some (Some [])


Well now we have a double option. That's not right, let's write a new pipeline operator to fix it.

In [8]:
(* Option.bind *)
let ( >>= ) opt f =
	match opt with
	| None -> None
	| Some x -> f x

val ( >>= ) : 'a option -> ('a -> 'b option) -> 'b option = <fun>


In [9]:
ListQueue.(empty |> enqueue 42 |> dequeue >>| enqueue 43 >>= dequeue)

- : int list option = Some []
